<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/335_SalesEnab_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 📊 Executive Reporting Utilities

**Turning orchestration into leadership-grade insight**

This module is where the orchestrator completes its job.

Up to this point, the agent has:

* prioritized leads
* analyzed customer needs
* tracked follow-ups
* nudged reps
* identified deal risk
* learned from historical outcomes

This reporting layer **compresses all of that complexity into executive-ready signals**.

---

## Why This Matters

Executives don’t want:

* raw logs
* long explanations
* per-lead detail

They want:

* pipeline health
* risk visibility
* performance clarity
* early warning signs

This module delivers exactly that — using **metrics leaders already trust**, backed by the agent’s rule-based reasoning.

---

## 🧠 `generate_pipeline_summary`

**Is the business healthy right now?**

This function produces a **one-page mental model of the pipeline**.

It answers questions like:

* How much business do we have?
* How much is real vs optimistic?
* How fast do we close?
* Where are we exposed?

---

### Core pipeline counts

The agent explicitly separates:

* active deals
* won deals
* lost deals

This avoids the common mistake of blending pipeline with outcomes.

Why this matters:

* Forecasts become honest
* Win rates are meaningful
* Leaders can distinguish momentum from results

---

### Total vs weighted pipeline value

```python
total_pipeline_value
weighted_pipeline_value
```

This distinction is critical.

* **Total pipeline** shows raw opportunity
* **Weighted pipeline** reflects execution reality

By applying stage-based weights, the agent:

* discounts early-stage optimism
* rewards real progress
* produces a more credible forecast

This mirrors how experienced CROs think — now systematized.

---

### Deal size & velocity

The agent calculates:

* average deal size
* average days to close (for won deals)

Why this matters:

* Sets expectations for forecasting
* Highlights changes in sales motion
* Reveals whether the team is speeding up or slowing down

Velocity is often more predictive than volume.

---

### Win rate & risk visibility

```python
win_rate
stalled_deals_count
at_risk_deals_count
```

These metrics surface **quality**, not just quantity.

Executives can immediately see:

* whether the team is converting
* how much pipeline needs intervention
* where attention should be focused

This is early-warning intelligence, not rear-view reporting.

---

## 🧑‍💼 `generate_rep_performance_summary`

**Who needs support — and who needs leverage?**

This function shifts focus from pipeline to **people**.

It creates a per-rep view that blends:

* output
* efficiency
* workload
* behavioral signals

---

### Performance metrics leaders recognize

For each rep, the agent reports:

* active deal count
* pipeline value
* close rate
* quota achievement
* year-to-date revenue

These are familiar metrics — which builds trust.

The difference is **how they’re interpreted**.

---

### Coaching need detection (rule-based, explainable)

```python
needs_coaching =
    quota_achievement < 70%
    OR close_rate < 30%
    OR too many nudges
```

This is a powerful design choice.

Instead of labeling reps subjectively, the agent uses:

* explicit thresholds
* observable behavior
* consistent logic

Why this matters:

* Coaching becomes fair
* Bias is reduced
* Managers gain confidence in the signal

This supports *coaching*, not punishment.

---

### Nudges as a performance signal

Including `nudges_count` is subtle — and very smart.

It acknowledges that:

* frequent nudging is a signal
* not all performance issues show up in revenue yet
* process breakdowns precede missed targets

This helps leaders intervene **before results suffer**.

---

### Top opportunities focus

By surfacing top 3 deals per rep, the agent:

* directs attention to where it matters most
* supports deal reviews
* avoids overwhelming managers

This is leverage, not micromanagement.

---

## Why This Reporting Layer Is Different

This module demonstrates that your orchestrator:

* 📊 reports outcomes leaders understand
* 🧠 embeds reasoning behind the numbers
* 🚨 surfaces risk early
* 🎯 supports coaching, not surveillance
* 💰 aligns directly with ROI and execution

Most AI systems stop at insight.
This one **closes the loop into leadership action**.

---

## How This Fits the Full Agent Story

With this reporting layer, you can confidently say:

> “This agent doesn’t just automate sales tasks — it provides executive-grade situational awareness.”

It ties together:

* prioritization
* nudging
* risk detection
* historical learning

And it does so in a way that:

* is auditable
* is configurable
* respects human judgment

---

## Portfolio Impact

Including this module signals that you:

* understand how executives think
* design for decision-makers, not just users
* build agents that *earn trust*

That’s a rare and valuable combination.


# Executive Reporting Utilities

In [ ]:
"""Executive Reporting Utilities

Generate pipeline summaries and rep performance metrics for executive reporting.
"""

from typing import Dict, Any, List, Optional
from datetime import datetime, timedelta


def generate_pipeline_summary(
    deals: List[Dict[str, Any]],
    stalled_deals: List[Dict[str, Any]],
    at_risk_deals: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Generate pipeline health summary.

    Args:
        deals: All deals in pipeline
        stalled_deals: List of stalled deals
        at_risk_deals: List of at-risk deals

    Returns:
        Pipeline summary dictionary
    """
    if not deals:
        return {
            "total_deals": 0,
            "active_deals": 0,
            "won_deals": 0,
            "lost_deals": 0,
            "total_pipeline_value": 0.0,
            "weighted_pipeline_value": 0.0,
            "average_deal_size": 0.0,
            "average_days_to_close": 0.0,
            "win_rate": 0.0,
            "stalled_deals_count": 0,
            "at_risk_deals_count": 0
        }

    # Count deals by status
    active_deals = [d for d in deals if d.get("status") == "active"]
    won_deals = [d for d in deals if d.get("status") == "won"]
    lost_deals = [d for d in deals if d.get("status") == "lost"]

    # Calculate pipeline values
    total_pipeline_value = sum(d.get("value_usd", 0.0) for d in active_deals)

    # Weighted pipeline value (using stage weights)
    stage_weights = {
        "Discovery": 0.1,
        "Qualification": 0.2,
        "Proposal": 0.4,
        "Negotiation": 0.7,
        "Closing": 0.9
    }
    weighted_pipeline_value = sum(
        d.get("value_usd", 0.0) * stage_weights.get(d.get("stage", "Discovery"), 0.1)
        for d in active_deals
    )

    # Average deal size
    all_deal_values = [d.get("value_usd", 0.0) for d in deals if d.get("value_usd", 0.0) > 0]
    average_deal_size = sum(all_deal_values) / len(all_deal_values) if all_deal_values else 0.0

    # Average days to close (for won deals)
    won_deals_with_close = [
        d for d in won_deals
        if d.get("closed_date") and d.get("created_date")
    ]
    if won_deals_with_close:
        days_to_close_list = []
        for deal in won_deals_with_close:
            try:
                created = datetime.fromisoformat(deal["created_date"].replace("Z", "+00:00"))
                closed = datetime.fromisoformat(deal["closed_date"].replace("Z", "+00:00"))
                days = (closed - created).days
                days_to_close_list.append(days)
            except (ValueError, KeyError):
                pass
        average_days_to_close = sum(days_to_close_list) / len(days_to_close_list) if days_to_close_list else 0.0
    else:
        average_days_to_close = 0.0

    # Win rate (won / (won + lost))
    total_closed = len(won_deals) + len(lost_deals)
    win_rate = len(won_deals) / total_closed if total_closed > 0 else 0.0

    return {
        "total_deals": len(deals),
        "active_deals": len(active_deals),
        "won_deals": len(won_deals),
        "lost_deals": len(lost_deals),
        "total_pipeline_value": total_pipeline_value,
        "weighted_pipeline_value": weighted_pipeline_value,
        "average_deal_size": average_deal_size,
        "average_days_to_close": average_days_to_close,
        "win_rate": win_rate,
        "stalled_deals_count": len(stalled_deals),
        "at_risk_deals_count": len(at_risk_deals)
    }


def generate_rep_performance_summary(
    sales_reps: List[Dict[str, Any]],
    deals: List[Dict[str, Any]],
    deals_lookup: Dict[str, Dict[str, Any]],
    rep_nudges: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Generate performance summary for each sales rep.

    Args:
        sales_reps: All sales reps
        deals: All deals
        deals_lookup: Lookup dictionary for deals by lead_id
        rep_nudges: List of nudges for reps

    Returns:
        List of rep performance summaries
    """
    summaries = []

    for rep in sales_reps:
        rep_id = rep.get("rep_id")
        if not rep_id:
            continue

        # Get deals for this rep
        rep_deals = [d for d in deals if d.get("assigned_rep_id") == rep_id]
        active_rep_deals = [d for d in rep_deals if d.get("status") == "active"]
        won_rep_deals = [d for d in rep_deals if d.get("status") == "won"]
        lost_rep_deals = [d for d in rep_deals if d.get("status") == "lost"]

        # Pipeline value
        pipeline_value = sum(d.get("value_usd", 0.0) for d in active_rep_deals)

        # Close rate (won / (won + lost))
        total_closed = len(won_rep_deals) + len(lost_rep_deals)
        close_rate = len(won_rep_deals) / total_closed if total_closed > 0 else 0.0

        # Quota achievement
        quota_usd = rep.get("quota_usd", 0.0)
        year_to_date_revenue = rep.get("year_to_date_revenue_usd", 0.0)
        quota_achievement = year_to_date_revenue / quota_usd if quota_usd > 0 else 0.0

        # Needs coaching (if has nudges or low performance)
        rep_nudges_count = len([n for n in rep_nudges if n.get("rep_id") == rep_id])
        needs_coaching = (
            quota_achievement < 0.7 or  # Below 70% of quota
            close_rate < 0.3 or  # Below 30% close rate
            rep_nudges_count > 5  # More than 5 nudges
        )

        # Top opportunities (top 3 active deals by value)
        top_opportunities = sorted(
            active_rep_deals,
            key=lambda d: d.get("value_usd", 0.0),
            reverse=True
        )[:3]
        top_opportunity_ids = [d.get("deal_id") for d in top_opportunities if d.get("deal_id")]

        summaries.append({
            "rep_id": rep_id,
            "rep_name": rep.get("name", "Unknown"),
            "active_deals": len(active_rep_deals),
            "pipeline_value": pipeline_value,
            "close_rate": close_rate,
            "quota_achievement": quota_achievement,
            "year_to_date_revenue": year_to_date_revenue,
            "quota_usd": quota_usd,
            "needs_coaching": needs_coaching,
            "top_opportunities": top_opportunity_ids,
            "nudges_count": rep_nudges_count
        })

    return summaries



# Report Generation Utilities

In [ ]:
"""Report Generation Utilities

Generate comprehensive sales enablement reports.
"""

from typing import Dict, Any, List, Optional
from datetime import datetime


def generate_sales_enablement_report(state: Dict[str, Any]) -> str:
    """
    Generate comprehensive sales enablement report.

    Args:
        state: Complete orchestrator state

    Returns:
        Markdown report string
    """
    goal = state.get("goal", {})
    prioritized_leads = state.get("prioritized_leads", [])
    top_priority_leads = state.get("top_priority_leads", [])
    customer_needs_analysis = state.get("customer_needs_analysis", [])
    outreach_recommendations = state.get("outreach_recommendations", [])
    follow_up_actions = state.get("follow_up_actions", [])
    rep_nudges = state.get("rep_nudges", [])
    deal_insights = state.get("deal_insights", [])
    stalled_deals = state.get("stalled_deals", [])
    at_risk_deals = state.get("at_risk_deals", [])
    win_patterns = state.get("win_patterns", [])
    loss_patterns = state.get("loss_patterns", [])
    pipeline_summary = state.get("pipeline_summary", {})
    rep_performance_summary = state.get("rep_performance_summary", [])
    errors = state.get("errors", [])

    report = f"""# Sales Enablement Report

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Objective:** {goal.get('objective', 'N/A')}

---

## Executive Summary

"""

    # Pipeline Summary
    if pipeline_summary:
        report += f"""### Pipeline Health

- **Total Deals:** {pipeline_summary.get('total_deals', 0)}
- **Active Deals:** {pipeline_summary.get('active_deals', 0)}
- **Won Deals:** {pipeline_summary.get('won_deals', 0)}
- **Lost Deals:** {pipeline_summary.get('lost_deals', 0)}
- **Total Pipeline Value:** ${pipeline_summary.get('total_pipeline_value', 0.0):,.0f}
- **Weighted Pipeline Value:** ${pipeline_summary.get('weighted_pipeline_value', 0.0):,.0f}
- **Average Deal Size:** ${pipeline_summary.get('average_deal_size', 0.0):,.0f}
- **Average Days to Close:** {pipeline_summary.get('average_days_to_close', 0.0):.1f} days
- **Win Rate:** {pipeline_summary.get('win_rate', 0.0)*100:.1f}%
- **Stalled Deals:** {pipeline_summary.get('stalled_deals_count', 0)}
- **At-Risk Deals:** {pipeline_summary.get('at_risk_deals_count', 0)}

"""

    # Rep Performance Summary
    if rep_performance_summary:
        report += "### Rep Performance\n\n"
        for rep_summary in rep_performance_summary:
            coaching_flag = "⚠️" if rep_summary.get("needs_coaching") else "✓"
            report += f"{coaching_flag} **{rep_summary.get('rep_name', 'Unknown')}** ({rep_summary.get('rep_id', 'N/A')})\n"
            report += f"   - Active Deals: {rep_summary.get('active_deals', 0)}\n"
            report += f"   - Pipeline Value: ${rep_summary.get('pipeline_value', 0.0):,.0f}\n"
            report += f"   - Close Rate: {rep_summary.get('close_rate', 0.0)*100:.1f}%\n"
            report += f"   - Quota Achievement: {rep_summary.get('quota_achievement', 0.0)*100:.1f}%\n"
            report += f"   - Nudges: {rep_summary.get('nudges_count', 0)}\n"
            report += "\n"

    report += "---\n\n"

    # Top Priority Leads
    if top_priority_leads:
        report += "## Top Priority Leads\n\n"
        for i, lead in enumerate(top_priority_leads[:10], 1):
            lead_id = lead.get("lead_id", "N/A")
            company_name = lead.get("company_name", "Unknown")
            priority_score = lead.get("priority_score", 0.0)
            report += f"{i}. **{company_name}** ({lead_id}) - Score: {priority_score:.1f}\n"
        report += "\n---\n\n"

    # Customer Needs Analysis
    if customer_needs_analysis:
        report += "## Customer Needs Analysis\n\n"
        for analysis in customer_needs_analysis[:5]:  # Top 5
            lead_id = analysis.get("lead_id", "N/A")
            pain_points = analysis.get("pain_points", [])
            buying_signals = analysis.get("buying_signals", [])
            product_fit = analysis.get("product_fit_score", 0.0)

            report += f"### Lead {lead_id}\n\n"
            if pain_points:
                report += f"- **Pain Points:** {', '.join(pain_points[:3])}\n"
            if buying_signals:
                report += f"- **Buying Signals:** {', '.join(buying_signals[:3])}\n"
            report += f"- **Product Fit Score:** {product_fit:.2f}\n\n"
        report += "---\n\n"

    # Outreach Recommendations
    if outreach_recommendations:
        report += "## Outreach Recommendations\n\n"
        for rec in outreach_recommendations[:5]:  # Top 5
            lead_id = rec.get("lead_id", "N/A")
            rep_id = rec.get("rep_id", "N/A")
            channel = rec.get("channel", "N/A")
            timing = rec.get("timing_suggestion", "N/A")
            report += f"- **{lead_id}** → {rep_id} via {channel} ({timing})\n"
        report += "\n---\n\n"

    # Follow-up Actions
    if follow_up_actions:
        overdue = [a for a in follow_up_actions if a.get("status") == "overdue"]
        due_soon = [a for a in follow_up_actions if a.get("status") == "due_soon"]

        report += "## Follow-up Actions\n\n"
        if overdue:
            report += f"### ⚠️ Overdue ({len(overdue)})\n\n"
            for action in overdue[:5]:
                lead_id = action.get("lead_id", "N/A")
                rep_id = action.get("rep_id", "N/A")
                action_type = action.get("action_type", "N/A")
                report += f"- {lead_id} → {rep_id}: {action_type}\n"
            report += "\n"

        if due_soon:
            report += f"### ⏰ Due Soon ({len(due_soon)})\n\n"
            for action in due_soon[:5]:
                lead_id = action.get("lead_id", "N/A")
                rep_id = action.get("rep_id", "N/A")
                action_type = action.get("action_type", "N/A")
                report += f"- {lead_id} → {rep_id}: {action_type}\n"
            report += "\n"

        report += "---\n\n"

    # Rep Nudges
    if rep_nudges:
        report += "## Rep Nudges\n\n"
        nudge_types = {}
        for nudge in rep_nudges:
            nudge_type = nudge.get("nudge_type", "unknown")
            nudge_types[nudge_type] = nudge_types.get(nudge_type, 0) + 1

        for nudge_type, count in nudge_types.items():
            report += f"- **{nudge_type.replace('_', ' ').title()}:** {count}\n"

        report += "\n### Sample Nudges\n\n"
        for nudge in rep_nudges[:5]:
            rep_id = nudge.get("rep_id", "N/A")
            message = nudge.get("message", "N/A")
            report += f"- **{rep_id}:** {message[:100]}...\n"
        report += "\n---\n\n"

    # Deal Insights
    if deal_insights:
        report += "## Deal Insights\n\n"

        if stalled_deals:
            report += f"### Stalled Deals ({len(stalled_deals)})\n\n"
            for deal in stalled_deals[:5]:
                deal_id = deal.get("deal_id", "N/A")
                lead_id = deal.get("lead_id", "N/A")
                days_in_stage = deal.get("days_in_stage", 0)
                report += f"- {deal_id} ({lead_id}): {days_in_stage} days in current stage\n"
            report += "\n"

        if at_risk_deals:
            report += f"### At-Risk Deals ({len(at_risk_deals)})\n\n"
            for deal in at_risk_deals[:5]:
                deal_id = deal.get("deal_id", "N/A")
                lead_id = deal.get("lead_id", "N/A")
                risk_factors = deal.get("risk_factors", [])
                report += f"- {deal_id} ({lead_id}): {', '.join(risk_factors[:2])}\n"
            report += "\n"

        report += "---\n\n"

    # Historical Insights
    if win_patterns or loss_patterns:
        report += "## Historical Insights\n\n"

        if win_patterns:
            report += "### Win Patterns\n\n"
            for pattern in win_patterns[:3]:
                pattern_type = pattern.get("pattern_type", "N/A")
                description = pattern.get("description", "N/A")
                frequency = pattern.get("frequency", 0.0)
                recommendation = pattern.get("recommendation", "N/A")
                report += f"- **{pattern_type.replace('_', ' ').title()}:** {description} ({frequency*100:.0f}% frequency)\n"
                report += f"  → Recommendation: {recommendation}\n"
            report += "\n"

        if loss_patterns:
            report += "### Loss Patterns\n\n"
            for pattern in loss_patterns[:3]:
                pattern_type = pattern.get("pattern_type", "N/A")
                description = pattern.get("description", "N/A")
                frequency = pattern.get("frequency", 0.0)
                recommendation = pattern.get("recommendation", "N/A")
                report += f"- **{pattern_type.replace('_', ' ').title()}:** {description} ({frequency*100:.0f}% frequency)\n"
                report += f"  → Recommendation: {recommendation}\n"
            report += "\n"

        report += "---\n\n"

    # Errors
    if errors:
        report += "## Errors\n\n"
        for error in errors:
            report += f"- ⚠️ {error}\n"
        report += "\n---\n\n"

    # Footer
    report += f"*Report generated by Sales Enablement Orchestrator Agent*\n"

    return report



# Test Suite for Phase 6: Executive Reporting & Report Generation

In [ ]:
"""Test Suite for Phase 6: Executive Reporting & Report Generation"""

from config import SalesEnablementOrchestratorState
from agents.sales_enablement.utilities.executive_reporting import (
    generate_pipeline_summary,
    generate_rep_performance_summary
)
from agents.sales_enablement.utilities.report_generation import (
    generate_sales_enablement_report
)
from agents.sales_enablement.utilities.data_loading import (
    load_deals,
    load_sales_reps,
    build_deals_lookup
)
from agents.sales_enablement.nodes import (
    executive_reporting_node,
    report_generation_node,
    data_loading_node,
    deal_insights_node,
    rep_nudging_node
)


def test_generate_pipeline_summary():
    """Test generate_pipeline_summary"""
    print("Testing generate_pipeline_summary...")

    deals = load_deals("agents/data", "deals.json")
    stalled_deals = [d for d in deals if d.get("days_in_stage", 0) > 20]
    at_risk_deals = [d for d in deals if d.get("sentiment", 0.5) < 0.3]

    summary = generate_pipeline_summary(deals, stalled_deals, at_risk_deals)

    assert "total_deals" in summary, "should have total_deals"
    assert "active_deals" in summary, "should have active_deals"
    assert "total_pipeline_value" in summary, "should have total_pipeline_value"
    assert "win_rate" in summary, "should have win_rate"
    assert summary["total_deals"] > 0, "should have deals"

    print(f"✅ generate_pipeline_summary test passed!")
    print(f"   Total deals: {summary['total_deals']}")
    print(f"   Active deals: {summary['active_deals']}")
    print(f"   Pipeline value: ${summary['total_pipeline_value']:,.0f}")
    print(f"   Win rate: {summary['win_rate']*100:.1f}%")
    print()


def test_generate_rep_performance_summary():
    """Test generate_rep_performance_summary"""
    print("Testing generate_rep_performance_summary...")

    from agents.sales_enablement.utilities.data_loading import load_sales_reps

    sales_reps = load_sales_reps("agents/data", "sales_reps.json")
    deals = load_deals("agents/data", "deals.json")
    deals_lookup = build_deals_lookup(deals)
    rep_nudges = []  # Empty for this test

    summaries = generate_rep_performance_summary(
        sales_reps,
        deals,
        deals_lookup,
        rep_nudges
    )

    assert len(summaries) > 0, "should generate summaries"
    assert "rep_id" in summaries[0], "should have rep_id"
    assert "pipeline_value" in summaries[0], "should have pipeline_value"
    assert "close_rate" in summaries[0], "should have close_rate"
    assert "quota_achievement" in summaries[0], "should have quota_achievement"

    print(f"✅ generate_rep_performance_summary test passed!")
    print(f"   Generated {len(summaries)} rep summaries")
    for summary in summaries[:3]:
        print(f"   - {summary['rep_name']}: {summary['active_deals']} deals, ${summary['pipeline_value']:,.0f} pipeline, {summary['quota_achievement']*100:.1f}% quota")
    print()


def test_executive_reporting_node():
    """Test executive_reporting_node"""
    print("Testing executive_reporting_node...")

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(deal_insights_node(state))
    state.update(rep_nudging_node(state))

    # Then generate executive reports
    result = executive_reporting_node(state)

    assert "pipeline_summary" in result, "should return pipeline_summary"
    assert "rep_performance_summary" in result, "should return rep_performance_summary"
    assert len(result["errors"]) == 0, "should have no errors"
    assert result["pipeline_summary"]["total_deals"] > 0, "should have deals"
    assert len(result["rep_performance_summary"]) > 0, "should have rep summaries"

    print(f"✅ executive_reporting_node test passed!")
    print(f"   Pipeline: {result['pipeline_summary']['active_deals']} active deals, ${result['pipeline_summary']['total_pipeline_value']:,.0f} value")
    print(f"   Reps: {len(result['rep_performance_summary'])} summaries")
    print()


def test_report_generation_node():
    """Test report_generation_node"""
    print("Testing report_generation_node...")

    # Build up full state
    from agents.sales_enablement.nodes import (
        goal_node, planning_node, lead_prioritization_node,
        customer_needs_analysis_node, outreach_generation_node,
        historical_insights_node
    )

    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(goal_node(state))
    state.update(planning_node(state))
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(customer_needs_analysis_node(state))
    state.update(outreach_generation_node(state))
    state.update(deal_insights_node(state))
    state.update(rep_nudging_node(state))
    state.update(historical_insights_node(state))
    state.update(executive_reporting_node(state))

    # Then generate report
    result = report_generation_node(state)

    assert "enablement_report" in result, "should return enablement_report"
    assert "report_file_path" in result, "should return report_file_path"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["enablement_report"]) > 0, "should have report content"
    assert result["report_file_path"].endswith(".md"), "should save as markdown"

    print(f"✅ report_generation_node test passed!")
    print(f"   Report length: {len(result['enablement_report'])} characters")
    print(f"   Saved to: {result['report_file_path']}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 6: Executive Reporting & Report Generation - Test Suite")
    print("=" * 60)
    print()

    test_generate_pipeline_summary()
    test_generate_rep_performance_summary()
    test_executive_reporting_node()
    test_report_generation_node()

    print("=" * 60)
    print("✅ All Phase 6 tests passed!")
    print("=" * 60)



# Sales Enablement Orchestrator Nodes

In [ ]:
"""Sales Enablement Orchestrator Nodes

This module contains all nodes for the Sales Enablement Orchestrator workflow.
Following the MVP-first approach: all nodes are rule-based (no LLM) for Phase 1-7.
"""

import json
from typing import Dict, Any
from datetime import datetime
from config import SalesEnablementOrchestratorState, SalesEnablementOrchestratorConfig
from agents.sales_enablement.utilities.data_loading import (
    load_leads,
    load_sales_reps,
    load_interactions,
    load_deals,
    load_signals,
    build_leads_lookup,
    build_reps_lookup,
    build_interactions_lookup,
    build_deals_lookup,
    build_signals_lookup
)
from agents.sales_enablement.utilities.lead_prioritization import (
    prioritize_leads,
    get_top_priority_leads
)
from agents.sales_enablement.utilities.customer_analysis import (
    analyze_all_customer_needs
)
from agents.sales_enablement.utilities.follow_up_tracking import (
    extract_follow_up_actions
)
from agents.sales_enablement.utilities.rep_nudging import (
    generate_all_rep_nudges
)
from agents.sales_enablement.utilities.deal_analysis import (
    detect_stalled_deals,
    detect_at_risk_deals,
    generate_all_deal_insights
)
from agents.sales_enablement.utilities.outreach_generation import (
    generate_all_outreach_recommendations
)
from agents.sales_enablement.utilities.historical_insights import (
    analyze_won_deals,
    analyze_lost_deals
)
from agents.sales_enablement.utilities.executive_reporting import (
    generate_pipeline_summary,
    generate_rep_performance_summary
)
from agents.sales_enablement.utilities.report_generation import (
    generate_sales_enablement_report
)
from toolshed.reporting import save_report


def goal_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for the Sales Enablement Orchestrator.

    This is a simple rule-based goal definition that sets the framework.
    No dependencies - this is the entry point.
    """
    lead_id = state.get("lead_id")
    rep_id = state.get("rep_id")
    focus_area = state.get("focus_area")

    # Build focus areas based on input
    if focus_area:
        focus_areas = [focus_area]
    else:
        focus_areas = [
            "lead_prioritization",
            "customer_needs_analysis",
            "outreach_generation",
            "follow_up_coordination",
            "rep_nudging",
            "deal_insights",
            "historical_insights",
            "executive_reporting"
        ]

    goal = {
        "objective": "Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights",
        "lead_id": lead_id,
        "rep_id": rep_id,
        "focus_areas": focus_areas,
        "scope": "all_leads" if not lead_id else "single_lead",
        "scope_reps": "all_reps" if not rep_id else "single_rep"
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    Only depends on goal_node.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    focus_areas = goal.get("focus_areas", [])

    # Build plan based on focus areas
    plan = []
    step_num = 1

    # Always start with data loading
    plan.append({
        "step": step_num,
        "name": "data_loading",
        "description": "Load all data files (leads, reps, interactions, deals, signals) and create lookup dictionaries",
        "dependencies": [],
        "outputs": ["leads", "sales_reps", "interactions", "deals", "signals", "leads_lookup", "reps_lookup", "interactions_lookup", "deals_lookup", "signals_lookup"]
    })
    step_num += 1

    # Lead prioritization (if in focus areas)
    if "lead_prioritization" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "lead_prioritization",
            "description": "Rank and prioritize leads based on intent, engagement, risk, and urgency",
            "dependencies": ["data_loading"],
            "outputs": ["prioritized_leads", "top_priority_leads"]
        })
        step_num += 1

    # Customer needs analysis (if in focus areas)
    if "customer_needs_analysis" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "customer_needs_analysis",
            "description": "Analyze customer pain points, buying signals, and objection likelihood",
            "dependencies": ["data_loading"],
            "outputs": ["customer_needs_analysis"]
        })
        step_num += 1

    # Outreach generation (if in focus areas)
    if "outreach_generation" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "outreach_generation",
            "description": "Generate personalized outreach recommendations with message drafts and timing",
            "dependencies": ["lead_prioritization", "customer_needs_analysis"],
            "outputs": ["outreach_recommendations"]
        })
        step_num += 1

    # Follow-up coordination (if in focus areas)
    if "follow_up_coordination" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "follow_up_coordination",
            "description": "Identify and track follow-up actions, detect overdue items",
            "dependencies": ["data_loading"],
            "outputs": ["follow_up_actions"]
        })
        step_num += 1

    # Rep nudging (if in focus areas)
    if "rep_nudging" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "rep_nudging",
            "description": "Generate nudges for sales reps based on overdue follow-ups, stalled deals, and high-priority leads",
            "dependencies": ["follow_up_coordination", "lead_prioritization", "deal_insights"],
            "outputs": ["rep_nudges"]
        })
        step_num += 1

    # Deal insights (if in focus areas)
    if "deal_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "deal_insights",
            "description": "Detect stalled deals, at-risk deals, and opportunities",
            "dependencies": ["data_loading"],
            "outputs": ["deal_insights", "stalled_deals", "at_risk_deals"]
        })
        step_num += 1

    # Historical insights (if in focus areas)
    if "historical_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "historical_insights",
            "description": "Analyze win/loss patterns from past deals to surface actionable insights",
            "dependencies": ["data_loading"],
            "outputs": ["win_patterns", "loss_patterns"]
        })
        step_num += 1

    # Executive reporting (if in focus areas)
    if "executive_reporting" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "executive_reporting",
            "description": "Generate pipeline summary and rep performance metrics",
            "dependencies": ["deal_insights", "lead_prioritization"],
            "outputs": ["pipeline_summary", "rep_performance_summary"]
        })
        step_num += 1

    # Always end with report generation
    plan.append({
        "step": step_num,
        "name": "report_generation",
        "description": "Generate final markdown report with all insights and recommendations",
        "dependencies": [p["name"] for p in plan if p["name"] != "report_generation"],
        "outputs": ["enablement_report", "report_file_path"]
    })

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading all data files and creating lookups.

    This node loads:
    - Leads
    - Sales Reps
    - Interactions
    - Deals
    - Signals

    And creates lookup dictionaries for fast access.
    """
    errors = state.get("errors", [])
    config = SalesEnablementOrchestratorConfig()

    try:
        # Load all data files
        leads = load_leads(config.data_dir, config.leads_file)
        sales_reps = load_sales_reps(config.data_dir, config.sales_reps_file)
        interactions = load_interactions(config.data_dir, config.interactions_file)
        deals = load_deals(config.data_dir, config.deals_file)
        signals = load_signals(config.data_dir, config.signals_file)

        # Build lookup dictionaries
        leads_lookup = build_leads_lookup(leads)
        reps_lookup = build_reps_lookup(sales_reps)
        interactions_lookup = build_interactions_lookup(interactions)
        deals_lookup = build_deals_lookup(deals)
        signals_lookup = build_signals_lookup(signals)

        return {
            "leads": leads,
            "sales_reps": sales_reps,
            "interactions": interactions,
            "deals": deals,
            "signals": signals,
            "leads_lookup": leads_lookup,
            "reps_lookup": reps_lookup,
            "interactions_lookup": interactions_lookup,
            "deals_lookup": deals_lookup,
            "signals_lookup": signals_lookup,
            "errors": errors
        }
    except FileNotFoundError as e:
        return {
            "errors": errors + [f"data_loading_node: File not found - {str(e)}"]
        }
    except json.JSONDecodeError as e:
        return {
            "errors": errors + [f"data_loading_node: Invalid JSON - {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: Unexpected error - {str(e)}"]
        }


def lead_prioritization_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Lead Prioritization Node: Rank and prioritize leads.

    Calculates priority scores for all leads and selects top N.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not leads:
        return {
            "errors": errors + ["lead_prioritization_node: leads required"]
        }

    try:
        # Prioritize all leads
        prioritized_leads = prioritize_leads(
            leads,
            signals_lookup,
            weights=config.priority_scoring_weights
        )

        # Get top N leads
        top_priority_leads = get_top_priority_leads(
            prioritized_leads,
            top_n=config.top_n_leads
        )

        return {
            "prioritized_leads": prioritized_leads,
            "top_priority_leads": top_priority_leads,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"lead_prioritization_node: {str(e)}"]
        }


def customer_needs_analysis_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Customer Needs Analysis Node: Analyze customer needs and pain points.

    Analyzes pain points, buying signals, objection likelihood, and product fit.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    interactions_lookup = state.get("interactions_lookup", {})
    deals_lookup = state.get("deals_lookup", {})

    if not leads:
        return {
            "errors": errors + ["customer_needs_analysis_node: leads required"]
        }

    try:
        # Analyze all customer needs
        customer_needs_analysis = analyze_all_customer_needs(
            leads,
            interactions_lookup,
            deals_lookup
        )

        return {
            "customer_needs_analysis": customer_needs_analysis,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"customer_needs_analysis_node: {str(e)}"]
        }


def follow_up_coordination_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Follow-up Coordination Node: Identify and track follow-up actions.

    Extracts follow-up actions from interactions and tracks their status.
    """
    errors = state.get("errors", [])
    interactions = state.get("interactions", [])
    config = SalesEnablementOrchestratorConfig()

    if not interactions:
        return {
            "errors": errors + ["follow_up_coordination_node: interactions required"]
        }

    try:
        # Extract follow-up actions
        follow_up_actions = extract_follow_up_actions(
            interactions,
            follow_up_overdue_days=config.follow_up_overdue_days
        )

        return {
            "follow_up_actions": follow_up_actions,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"follow_up_coordination_node: {str(e)}"]
        }


def deal_insights_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Deal Insights Node: Detect stalled deals, at-risk deals, and opportunities.
    """
    errors = state.get("errors", [])
    deals = state.get("deals", [])
    interactions_lookup = state.get("interactions_lookup", {})
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not deals:
        return {
            "errors": errors + ["deal_insights_node: deals required"]
        }

    try:
        # Generate all deal insights
        deal_insights = generate_all_deal_insights(
            deals,
            interactions_lookup,
            signals_lookup,
            config
        )

        # Separate by type
        stalled_deals = [i for i in deal_insights if i.get("insight_type") == "stalled"]
        at_risk_deals = [i for i in deal_insights if i.get("insight_type") == "at_risk"]

        return {
            "deal_insights": deal_insights,
            "stalled_deals": stalled_deals,
            "at_risk_deals": at_risk_deals,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"deal_insights_node: {str(e)}"]
        }


def rep_nudging_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Rep Nudging Node: Generate nudges for sales reps.

    Generates nudges based on:
    - Overdue follow-ups
    - Stalled deals
    - High-priority leads with no interaction
    - Deals at risk
    """
    errors = state.get("errors", [])
    follow_up_actions = state.get("follow_up_actions", [])
    stalled_deals = state.get("stalled_deals", [])
    at_risk_deals = state.get("at_risk_deals", [])
    top_priority_leads = state.get("top_priority_leads", [])
    leads_lookup = state.get("leads_lookup", {})
    interactions_lookup = state.get("interactions_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    try:
        # Generate all nudges
        rep_nudges = generate_all_rep_nudges(
            follow_up_actions,
            stalled_deals,
            at_risk_deals,
            top_priority_leads,
            leads_lookup,
            interactions_lookup,
            config
        )

        return {
            "rep_nudges": rep_nudges,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"rep_nudging_node: {str(e)}"]
        }


def outreach_generation_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Outreach Generation Node: Generate personalized outreach recommendations.

    Creates outreach recommendations with message drafts, timing, and channel selection.
    """
    errors = state.get("errors", [])
    prioritized_leads = state.get("prioritized_leads", [])
    sales_reps = state.get("sales_reps", [])
    interactions_lookup = state.get("interactions_lookup", {})
    customer_needs_analysis = state.get("customer_needs_analysis", [])
    deals_lookup = state.get("deals_lookup", {})
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not prioritized_leads:
        return {
            "errors": errors + ["outreach_generation_node: prioritized_leads required"]
        }

    try:
        # Generate outreach recommendations for top N leads
        outreach_recommendations = generate_all_outreach_recommendations(
            prioritized_leads,
            sales_reps,
            interactions_lookup,
            customer_needs_analysis,
            deals_lookup,
            signals_lookup,
            top_n=config.top_n_leads
        )

        return {
            "outreach_recommendations": outreach_recommendations,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"outreach_generation_node: {str(e)}"]
        }


def historical_insights_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Historical Insights Node: Analyze win/loss patterns from past deals.

    Identifies patterns from won and lost deals to surface actionable insights.
    """
    errors = state.get("errors", [])
    deals = state.get("deals", [])
    interactions_lookup = state.get("interactions_lookup", {})

    if not deals:
        return {
            "errors": errors + ["historical_insights_node: deals required"]
        }

    try:
        # Analyze won deals
        win_patterns = analyze_won_deals(deals, interactions_lookup)

        # Analyze lost deals
        loss_patterns = analyze_lost_deals(deals, interactions_lookup)

        return {
            "win_patterns": win_patterns,
            "loss_patterns": loss_patterns,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"historical_insights_node: {str(e)}"]
        }


def executive_reporting_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Executive Reporting Node: Generate pipeline and rep performance summaries.

    Creates executive-level summaries for pipeline health and rep performance.
    """
    errors = state.get("errors", [])
    deals = state.get("deals", [])
    stalled_deals = state.get("stalled_deals", [])
    at_risk_deals = state.get("at_risk_deals", [])
    sales_reps = state.get("sales_reps", [])
    deals_lookup = state.get("deals_lookup", {})
    rep_nudges = state.get("rep_nudges", [])

    if not deals:
        return {
            "errors": errors + ["executive_reporting_node: deals required"]
        }

    try:
        # Generate pipeline summary
        pipeline_summary = generate_pipeline_summary(
            deals,
            stalled_deals,
            at_risk_deals
        )

        # Generate rep performance summary
        rep_performance_summary = generate_rep_performance_summary(
            sales_reps,
            deals,
            deals_lookup,
            rep_nudges
        )

        return {
            "pipeline_summary": pipeline_summary,
            "rep_performance_summary": rep_performance_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"executive_reporting_node: {str(e)}"]
        }


def report_generation_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Report Generation Node: Generate and save the final sales enablement report.

    Creates a comprehensive markdown report and saves it to disk.
    """
    errors = state.get("errors", [])
    config = SalesEnablementOrchestratorConfig()

    try:
        # Generate the report
        report_content = generate_sales_enablement_report(state)

        # Save the report
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_file_path = save_report(
            report_content=report_content,
            report_id=timestamp,
            reports_dir=config.reports_dir,
            prefix="sales_enablement"
        )

        return {
            "enablement_report": report_content,
            "report_file_path": report_file_path,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: {str(e)}"]
        }



# Test Phase 1: Foundation Nodes (Goal & Planning)

In [ ]:
"""Test Phase 1: Foundation Nodes (Goal & Planning)

Test goal_node and planning_node independently.
"""

from agents.sales_enablement.nodes import goal_node, planning_node
from config import SalesEnablementOrchestratorState


def test_goal_node():
    """Test goal_node with no input"""
    print("Testing goal_node...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result, "goal_node should return 'goal'"
    assert result["goal"]["objective"] is not None, "goal should have objective"
    assert len(result["goal"]["focus_areas"]) > 0, "goal should have focus_areas"
    assert result["goal"]["scope"] == "all_leads", "scope should be 'all_leads' when no lead_id"

    print(f"✅ goal_node test passed!")
    print(f"   Objective: {result['goal']['objective'][:60]}...")
    print(f"   Focus areas: {len(result['goal']['focus_areas'])} areas")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_lead_id():
    """Test goal_node with specific lead_id"""
    print("Testing goal_node with lead_id...")

    state: SalesEnablementOrchestratorState = {
        "lead_id": "L-001",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["lead_id"] == "L-001", "goal should include lead_id"
    assert result["goal"]["scope"] == "single_lead", "scope should be 'single_lead'"

    print(f"✅ goal_node with lead_id test passed!")
    print(f"   Lead ID: {result['goal']['lead_id']}")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_focus_area():
    """Test goal_node with specific focus_area"""
    print("Testing goal_node with focus_area...")

    state: SalesEnablementOrchestratorState = {
        "focus_area": "lead_prioritization",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["focus_areas"] == ["lead_prioritization"], "focus_areas should match input"

    print(f"✅ goal_node with focus_area test passed!")
    print(f"   Focus areas: {result['goal']['focus_areas']}")
    print()


def test_planning_node():
    """Test planning_node with goal"""
    print("Testing planning_node...")

    state: SalesEnablementOrchestratorState = {
        "goal": {
            "objective": "Test objective",
            "focus_areas": ["lead_prioritization", "outreach_generation"]
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result, "planning_node should return 'plan'"
    assert len(result["plan"]) > 0, "plan should have steps"

    # Check that plan includes data_loading (always first)
    assert result["plan"][0]["name"] == "data_loading", "First step should be data_loading"

    # Check that plan includes requested focus areas
    plan_names = [step["name"] for step in result["plan"]]
    assert "lead_prioritization" in plan_names, "plan should include lead_prioritization"
    assert "outreach_generation" in plan_names, "plan should include outreach_generation"

    # Check that plan ends with report_generation
    assert result["plan"][-1]["name"] == "report_generation", "Last step should be report_generation"

    print(f"✅ planning_node test passed!")
    print(f"   Total steps: {len(result['plan'])}")
    print(f"   Steps: {[step['name'] for step in result['plan']]}")
    print()


def test_planning_node_without_goal():
    """Test planning_node error handling when goal is missing"""
    print("Testing planning_node error handling...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result, "planning_node should return errors"
    assert len(result["errors"]) > 0, "should have error when goal is missing"
    assert "goal is required" in result["errors"][0], "error should mention goal"

    print(f"✅ planning_node error handling test passed!")
    print(f"   Error: {result['errors'][0]}")
    print()


def test_goal_and_planning_chain():
    """Test goal_node → planning_node chain"""
    print("Testing goal_node → planning_node chain...")

    # Start with empty state
    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    # Run goal_node (merge result into state)
    goal_result = goal_node(state)
    state.update(goal_result)
    assert "goal" in state, "goal should be in state after goal_node"

    # Run planning_node (merge result into state)
    plan_result = planning_node(state)
    state.update(plan_result)
    assert "plan" in state, "plan should be in state after planning_node"
    assert len(state["errors"]) == 0, "should have no errors in successful chain"

    print(f"✅ goal_node → planning_node chain test passed!")
    print(f"   Goal objective: {state['goal']['objective'][:50]}...")
    print(f"   Plan steps: {len(state['plan'])}")
    print()


def test_data_loading_node():
    """Test data_loading_node"""
    print("Testing data_loading_node...")

    from agents.sales_enablement.nodes import data_loading_node

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = data_loading_node(state)

    assert "leads" in result, "data_loading_node should return 'leads'"
    assert "sales_reps" in result, "data_loading_node should return 'sales_reps'"
    assert "interactions" in result, "data_loading_node should return 'interactions'"
    assert "deals" in result, "data_loading_node should return 'deals'"
    assert "signals" in result, "data_loading_node should return 'signals'"
    assert "leads_lookup" in result, "data_loading_node should return 'leads_lookup'"
    assert "reps_lookup" in result, "data_loading_node should return 'reps_lookup'"
    assert "interactions_lookup" in result, "data_loading_node should return 'interactions_lookup'"
    assert "deals_lookup" in result, "data_loading_node should return 'deals_lookup'"
    assert "signals_lookup" in result, "data_loading_node should return 'signals_lookup'"
    assert len(result["errors"]) == 0, "should have no errors"

    # Verify data loaded correctly
    assert len(result["leads"]) == 20, "should load 20 leads"
    assert len(result["sales_reps"]) == 4, "should load 4 reps"
    assert len(result["interactions"]) == 12, "should load 12 interactions"
    assert len(result["deals"]) == 15, "should load 15 deals"
    assert len(result["signals"]) == 20, "should load 20 signals"

    # Verify lookups work
    assert "L-001" in result["leads_lookup"], "leads_lookup should contain L-001"
    assert "SR-01" in result["reps_lookup"], "reps_lookup should contain SR-01"

    print(f"✅ data_loading_node test passed!")
    print(f"   Loaded: {len(result['leads'])} leads, {len(result['sales_reps'])} reps, {len(result['interactions'])} interactions, {len(result['deals'])} deals, {len(result['signals'])} signals")
    print(f"   Lookups: {len(result['leads_lookup'])} leads, {len(result['reps_lookup'])} reps, {len(result['interactions_lookup'])} leads with interactions, {len(result['deals_lookup'])} leads with deals")
    print()

    return result


def test_lead_prioritization_node():
    """Test lead_prioritization_node"""
    print("Testing lead_prioritization_node...")

    from agents.sales_enablement.nodes import lead_prioritization_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then prioritize
    result = lead_prioritization_node(state)

    assert "prioritized_leads" in result, "should return prioritized_leads"
    assert "top_priority_leads" in result, "should return top_priority_leads"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["prioritized_leads"]) == 20, "should prioritize all 20 leads"
    assert len(result["top_priority_leads"]) == 10, "should return top 10 leads"
    assert result["prioritized_leads"][0]["priority_score"] >= result["prioritized_leads"][-1]["priority_score"], "should be sorted"

    print(f"✅ lead_prioritization_node test passed!")
    print(f"   Prioritized: {len(result['prioritized_leads'])} leads")
    print(f"   Top {len(result['top_priority_leads'])} leads:")
    for i, lead in enumerate(result["top_priority_leads"][:3], 1):
        print(f"     {i}. {lead['lead_id']} - {lead['company_name']} (Score: {lead['priority_score']:.2f})")
    print()


def test_customer_needs_analysis_node():
    """Test customer_needs_analysis_node"""
    print("Testing customer_needs_analysis_node...")

    from agents.sales_enablement.nodes import customer_needs_analysis_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze
    result = customer_needs_analysis_node(state)

    assert "customer_needs_analysis" in result, "should return customer_needs_analysis"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["customer_needs_analysis"]) == 20, "should analyze all 20 leads"
    assert "buying_signals" in result["customer_needs_analysis"][0], "should have buying_signals"
    assert "product_fit" in result["customer_needs_analysis"][0], "should have product_fit"

    print(f"✅ customer_needs_analysis_node test passed!")
    print(f"   Analyzed: {len(result['customer_needs_analysis'])} leads")
    sample = result["customer_needs_analysis"][0]
    print(f"   Sample: {sample['lead_id']} - Product fit: {sample['product_fit']}, Buying signals: {len(sample['buying_signals'])}")
    print()


def test_follow_up_coordination_node():
    """Test follow_up_coordination_node"""
    print("Testing follow_up_coordination_node...")

    from agents.sales_enablement.nodes import follow_up_coordination_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then coordinate follow-ups
    result = follow_up_coordination_node(state)

    assert "follow_up_actions" in result, "should return follow_up_actions"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["follow_up_actions"]) > 0, "should find some follow-ups"
    assert "status" in result["follow_up_actions"][0], "should have status"

    print(f"✅ follow_up_coordination_node test passed!")
    print(f"   Follow-up actions: {len(result['follow_up_actions'])}")
    overdue = [f for f in result["follow_up_actions"] if f.get("status") == "overdue"]
    print(f"   Overdue: {len(overdue)}")
    print()


def test_deal_insights_node():
    """Test deal_insights_node"""
    print("Testing deal_insights_node...")

    from agents.sales_enablement.nodes import deal_insights_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze deals
    result = deal_insights_node(state)

    assert "deal_insights" in result, "should return deal_insights"
    assert "stalled_deals" in result, "should return stalled_deals"
    assert "at_risk_deals" in result, "should return at_risk_deals"
    assert len(result["errors"]) == 0, "should have no errors"

    print(f"✅ deal_insights_node test passed!")
    print(f"   Total insights: {len(result['deal_insights'])}")
    print(f"   Stalled deals: {len(result['stalled_deals'])}")
    print(f"   At-risk deals: {len(result['at_risk_deals'])}")
    print()


def test_rep_nudging_node():
    """Test rep_nudging_node"""
    print("Testing rep_nudging_node...")

    from agents.sales_enablement.nodes import (
        rep_nudging_node, data_loading_node, lead_prioritization_node,
        follow_up_coordination_node, deal_insights_node
    )

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(follow_up_coordination_node(state))
    state.update(deal_insights_node(state))

    # Then generate nudges
    result = rep_nudging_node(state)

    assert "rep_nudges" in result, "should return rep_nudges"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["rep_nudges"]) > 0, "should generate some nudges"
    assert "nudge_type" in result["rep_nudges"][0], "should have nudge_type"
    assert "message" in result["rep_nudges"][0], "should have message"

    print(f"✅ rep_nudging_node test passed!")
    print(f"   Generated {len(result['rep_nudges'])} nudges")

    # Show nudge types
    nudge_types = {}
    for nudge in result["rep_nudges"]:
        nudge_type = nudge.get("nudge_type", "unknown")
        nudge_types[nudge_type] = nudge_types.get(nudge_type, 0) + 1

    print(f"   Nudge types: {nudge_types}")
    if result["rep_nudges"]:
        print(f"   Sample nudge: {result['rep_nudges'][0]['message'][:60]}...")
    print()


def test_outreach_generation_node():
    """Test outreach_generation_node"""
    print("Testing outreach_generation_node...")

    from agents.sales_enablement.nodes import (
        outreach_generation_node, data_loading_node, lead_prioritization_node,
        customer_needs_analysis_node
    )

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(customer_needs_analysis_node(state))

    # Then generate outreach
    result = outreach_generation_node(state)

    assert "outreach_recommendations" in result, "should return outreach_recommendations"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["outreach_recommendations"]) > 0, "should generate some recommendations"
    assert "message_draft" in result["outreach_recommendations"][0], "should have message_draft"
    assert "channel" in result["outreach_recommendations"][0], "should have channel"

    print(f"✅ outreach_generation_node test passed!")
    print(f"   Generated {len(result['outreach_recommendations'])} outreach recommendations")
    sample = result["outreach_recommendations"][0]
    print(f"   Sample: {sample['lead_id']} -> {sample['rep_id']} via {sample['channel']}")
    print(f"   Message preview: {sample['message_draft'][:50]}...")
    print()


def test_historical_insights_node():
    """Test historical_insights_node"""
    print("Testing historical_insights_node...")

    from agents.sales_enablement.nodes import historical_insights_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze historical patterns
    result = historical_insights_node(state)

    assert "win_patterns" in result, "should return win_patterns"
    assert "loss_patterns" in result, "should return loss_patterns"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["win_patterns"]) > 0, "should find some win patterns"

    print(f"✅ historical_insights_node test passed!")
    print(f"   Win patterns: {len(result['win_patterns'])}")
    print(f"   Loss patterns: {len(result['loss_patterns'])}")
    if result["win_patterns"]:
        print(f"   Sample win pattern: {result['win_patterns'][0]['pattern_type']} - {result['win_patterns'][0]['description'][:50]}...")
    print()


def test_executive_reporting_node():
    """Test executive_reporting_node"""
    print("Testing executive_reporting_node...")

    from agents.sales_enablement.nodes import (
        executive_reporting_node, data_loading_node, deal_insights_node, rep_nudging_node
    )

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(deal_insights_node(state))
    state.update(rep_nudging_node(state))

    # Then generate executive reports
    result = executive_reporting_node(state)

    assert "pipeline_summary" in result, "should return pipeline_summary"
    assert "rep_performance_summary" in result, "should return rep_performance_summary"
    assert len(result["errors"]) == 0, "should have no errors"

    print(f"✅ executive_reporting_node test passed!")
    print(f"   Pipeline: {result['pipeline_summary']['active_deals']} active deals")
    print(f"   Reps: {len(result['rep_performance_summary'])} summaries")
    print()


def test_report_generation_node():
    """Test report_generation_node"""
    print("Testing report_generation_node...")

    from agents.sales_enablement.nodes import (
        goal_node, planning_node, lead_prioritization_node,
        customer_needs_analysis_node, outreach_generation_node,
        historical_insights_node, executive_reporting_node
    )

    # Build up full state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(goal_node(state))
    state.update(planning_node(state))
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(customer_needs_analysis_node(state))
    state.update(outreach_generation_node(state))
    state.update(deal_insights_node(state))
    state.update(rep_nudging_node(state))
    state.update(historical_insights_node(state))
    state.update(executive_reporting_node(state))

    # Then generate report
    from agents.sales_enablement.nodes import report_generation_node
    result = report_generation_node(state)

    assert "enablement_report" in result, "should return enablement_report"
    assert "report_file_path" in result, "should return report_file_path"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["enablement_report"]) > 0, "should have report content"

    print(f"✅ report_generation_node test passed!")
    print(f"   Report length: {len(result['enablement_report'])} characters")
    print(f"   Saved to: {result['report_file_path']}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 1-5: All Nodes - Test Suite")
    print("=" * 60)
    print()

    test_goal_node()
    test_goal_node_with_lead_id()
    test_goal_node_with_focus_area()
    test_planning_node()
    test_planning_node_without_goal()
    test_goal_and_planning_chain()
    test_data_loading_node()
    test_lead_prioritization_node()
    test_customer_needs_analysis_node()
    test_follow_up_coordination_node()
    test_deal_insights_node()
    test_rep_nudging_node()
    test_outreach_generation_node()
    test_historical_insights_node()
    test_executive_reporting_node()
    test_report_generation_node()

    print("=" * 60)
    print("✅ All Phase 1-6 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python test_sales_enablement_phase1.py
============================================================
Phase 1-5: All Nodes - Test Suite
============================================================

Testing goal_node...
✅ goal_node test passed!
   Objective: Enable sales team performance by prioritizing leads, analyzi...
   Focus areas: 8 areas
   Scope: all_leads

Testing goal_node with lead_id...
✅ goal_node with lead_id test passed!
   Lead ID: L-001
   Scope: single_lead

Testing goal_node with focus_area...
✅ goal_node with focus_area test passed!
   Focus areas: ['lead_prioritization']

Testing planning_node...
✅ planning_node test passed!
   Total steps: 4
   Steps: ['data_loading', 'lead_prioritization', 'outreach_generation', 'report_generation']

Testing planning_node error handling...
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

Testing goal_node → planning_node chain...
✅ goal_node → planning_node chain test passed!
   Goal objective: Enable sales team performance by prioritizing lead...
   Plan steps: 10

Testing data_loading_node...
✅ data_loading_node test passed!
   Loaded: 20 leads, 4 reps, 12 interactions, 15 deals, 20 signals
   Lookups: 20 leads, 4 reps, 9 leads with interactions, 15 leads with deals

Testing lead_prioritization_node...
✅ lead_prioritization_node test passed!
   Prioritized: 20 leads
   Top 10 leads:
     1. L-020 - Orion Aerospace (Score: 86.60)
     2. L-003 - Apex Manufacturing (Score: 85.50)
     3. L-011 - Atlas Freight (Score: 83.00)

Testing customer_needs_analysis_node...
✅ customer_needs_analysis_node test passed!
   Analyzed: 20 leads
   Sample: L-001 - Product fit: 0.63, Buying signals: 1

Testing follow_up_coordination_node...
✅ follow_up_coordination_node test passed!
   Follow-up actions: 7
   Overdue: 7

Testing deal_insights_node...
✅ deal_insights_node test passed!
   Total insights: 11
   Stalled deals: 2
   At-risk deals: 5

Testing rep_nudging_node...
✅ rep_nudging_node test passed!
   Generated 24 nudges
   Nudge types: {'follow_up_due': 7, 'stalled_deal': 2, 'high_priority_lead': 10, 'deal_at_risk': 5}
   Sample nudge: ⚠️ Follow-up with Northstar Logistics (L-001) is overdue. Yo...

Testing outreach_generation_node...
✅ outreach_generation_node test passed!
   Generated 10 outreach recommendations
   Sample: L-020 -> SR-04 via email
   Message preview: Hi VP Strategy at Orion Aerospace,

I noticed that...

Testing historical_insights_node...
✅ historical_insights_node test passed!
   Win patterns: 4
   Loss patterns: 4
   Sample win pattern: interaction_frequency - Won deals had 1.0 interactions on average (range: ...

Testing executive_reporting_node...
✅ executive_reporting_node test passed!
   Pipeline: 11 active deals
   Reps: 4 summaries

Testing report_generation_node...
✅ report_generation_node test passed!
   Report length: 4700 characters
   Saved to: output/sales_enablement_reports/sales_enablement_20251229_170954_20251229_170954.md

============================================================
✅ All Phase 1-6 tests passed!
============================================================


# Sales Enablement Report

**Generated:** 2025-12-29 17:09:54  
**Objective:** Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights

---

## Executive Summary

### Pipeline Health

- **Total Deals:** 15
- **Active Deals:** 11
- **Won Deals:** 2
- **Lost Deals:** 2
- **Total Pipeline Value:** $0
- **Weighted Pipeline Value:** $0
- **Average Deal Size:** $0
- **Average Days to Close:** 0.0 days
- **Win Rate:** 50.0%
- **Stalled Deals:** 2
- **At-Risk Deals:** 5

### Rep Performance

⚠️ **Alex Morgan** (SR-01)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 81.0%
   - Nudges: 0

⚠️ **Jordan Lee** (SR-02)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 65.3%
   - Nudges: 0

⚠️ **Priya Shah** (SR-03)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 95.0%
   - Nudges: 0

⚠️ **Miguel Alvarez** (SR-04)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 54.0%
   - Nudges: 0

---

## Top Priority Leads

1. **Orion Aerospace** (L-020) - Score: 86.6
2. **Apex Manufacturing** (L-003) - Score: 85.5
3. **Atlas Freight** (L-011) - Score: 83.0
4. **NovaEnergy Solutions** (L-006) - Score: 81.8
5. **OmniPharma** (L-015) - Score: 77.5
6. **Northstar Logistics** (L-001) - Score: 74.5
7. **Vertex Consulting** (L-014) - Score: 71.8
8. **Skyline Retail Group** (L-005) - Score: 70.2
9. **Horizon AgriTech** (L-018) - Score: 69.3
10. **ClearWave Health** (L-002) - Score: 62.6

---

## Customer Needs Analysis

### Lead L-001

- **Pain Points:** manual reporting, forecast inaccuracy
- **Buying Signals:** positive engagement
- **Product Fit Score:** 0.00

### Lead L-002

- **Pain Points:** data silos, slow reporting
- **Product Fit Score:** 0.00

### Lead L-003

- **Pain Points:** cost overruns, margin pressure
- **Buying Signals:** proposal requested, positive engagement, pricing discussed
- **Product Fit Score:** 0.00

### Lead L-004

- **Pain Points:** budget forecasting, grant reporting
- **Product Fit Score:** 0.00

### Lead L-005

- **Pain Points:** inventory forecasting, seasonality
- **Product Fit Score:** 0.00

---

## Outreach Recommendations

- **L-020** → SR-04 via email (N/A)
- **L-003** → SR-01 via email (N/A)
- **L-011** → SR-01 via email (N/A)
- **L-006** → SR-01 via email (N/A)
- **L-015** → SR-03 via call (N/A)

---

## Rep Nudges

- **Stalled Deal:** 2
- **High Priority Lead:** 10
- **Deal At Risk:** 5

### Sample Nudges

- **None:** ⏸️ Deal D-004 (Skyline Retail Group, $0) has been in Negotiation stage for 21 days. Consider advanci...
- **None:** ⏸️ Deal D-008 (Ironclad Construction, $0) has been in Proposal stage for 22 days. Consider advancing...
- **None:** ⭐ High-priority lead: Orion Aerospace (L-020) has priority score 86.6. assign senior rep and initiat...
- **None:** ⭐ High-priority lead: Apex Manufacturing (L-003) has priority score 85.5. follow up on proposal....
- **None:** ⭐ High-priority lead: Atlas Freight (L-011) has priority score 83.0. handoff to onboarding....

---

## Deal Insights

### Stalled Deals (2)

- D-004 (L-005): 21 days in current stage
- D-008 (L-009): 22 days in current stage

### At-Risk Deals (5)

- D-002 (L-002): pricing sensitivity
- D-004 (L-005): discount pressure, negative sentiment in recent interactions
- D-007 (L-008): budget uncertainty
- D-008 (L-009): timeline risk
- D-014 (L-015): legal review

---

## Historical Insights

### Win Patterns

- **Interaction Frequency:** Won deals had 1.0 interactions on average (range: 1-1) (100% frequency)
  → Recommendation: Aim for 1+ interactions for similar deals
- **Positive Sentiment:** 0/2 won deals had 2+ positive interactions (0% frequency)
  → Recommendation: Focus on building positive engagement early in the sales cycle
- **Pricing Discussion:** 0/2 won deals discussed pricing (0% frequency)
  → Recommendation: Introduce pricing discussion early for qualified leads

### Loss Patterns

- **Negative Sentiment:** 1/2 lost deals had negative sentiment (50% frequency)
  → Recommendation: Address negative sentiment immediately when detected
- **Risk Flags:** Most common risk flag in lost deals: 'pricing loss' (1 occurrences) (50% frequency)
  → Recommendation: Monitor and address 'pricing loss' risk flags proactively
- **Competition:** Most common competitor in lost deals: 'VendorX' (1 occurrences) (50% frequency)
  → Recommendation: Develop competitive differentiation strategy against 'VendorX'

---

*Report generated by Sales Enablement Orchestrator Agent*
